In [127]:
import MLDataset
import paths
import evaluate_model
from datetime import datetime

lasso_dataset = MLDataset.MLDataset(f'{paths.ML_DATASETS_FOLDER}/New48sectors.pkl.gz')
lgbm_dataset = MLDataset.MLDataset(f'{paths.ML_DATASETS_FOLDER}/LGBM_after_New48sectors.pkl.gz')
en_dataset = MLDataset.MLDataset(f'{paths.ML_DATASETS_FOLDER}/LGBM_New48sectors_Ensembled.pkl.gz')


インスタンス情報を読み込みました。
インスタンス情報を読み込みました。
インスタンス情報を読み込みました。


In [128]:
import pandas as pd
df = pd.concat([lasso_dataset.pred_result_df, lgbm_dataset.pred_result_df[['Pred']], en_dataset.pred_result_df[['Pred']]], axis=1)
df = df[df.index.get_level_values(0)>=datetime(2022,1,1)]
df.columns = ['Act', 'LASSO', 'LGBM', 'En_rank']
for x in ['Act', 'LASSO', 'LGBM']:
    df[f'{x}_rank'] = df.groupby('Date')[x].rank(ascending=True)
df = df[['Act_rank', 'LASSO_rank', 'LGBM_rank', 'En_rank']]
df.corr()

,Act_rank,LASSO_rank,LGBM_rank,En_rank
Act_rank,1.000000,0.040645,0.048550,0.044487
LASSO_rank,0.040645,1.000000,0.632555,0.980882
LGBM_rank,0.048550,0.632555,1.000000,0.754024
En_rank,0.044487,0.980882,0.754024,1.000000


In [129]:
ml_dataset = en_dataset
model_obj = evaluate_model.LongShortModel(model_name = 'DJ_residuals', pred_result_df = ml_dataset.pred_result_df, 
                                          raw_target_df = ml_dataset.raw_target_df,
                                          start_date = datetime(2022, 1, 1), end_date = datetime.today(), 
                                          bin_num = 5, top_slope=1)
model_obj.display_result()

h:\マイドライブ\python\Lib\site-packages\numpy\_core\fromnumeric.py:84: FutureWarning:

The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)

h:\マイドライブ\python\Lib\site-packages\numpy\_core\fromnumeric.py:84: FutureWarning:

The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)



DJ_residualsの読み込みが完了しました。
全期間
母集団の推定（99.7%）
平均リターン: 0.1568% (worst: 0.0954%, best: 0.2182%)
標準偏差: 0.005397 (worst:0.005863, best:0.004993)
-----------------------------------------
モデルのスペック
シャープレシオ: 0.290549 (worst:0.162678, best:0.437016)
最大ドローダウン: 4.1791% (worst:8.1096%, best:2.5708%)
最大ドローダウン日数: 107日 (worst:340日, best:47日)
----------------------------------------------------------------------------------
直近1年
母集団の推定（99.7%）
平均リターン: 0.1457% (worst: 0.0417%, best: 0.2497%)
標準偏差: 0.00551 (worst:0.006338, best:0.004859)
-----------------------------------------
モデルのスペック
シャープレシオ: 0.264491 (worst:0.065805, best:0.513979)
最大ドローダウン: 4.6869% (worst:21.6703%, best:2.127%)
最大ドローダウン日数: 129日 (worst:2078日, best:34日)
----------------------------------------------------------------------------------
直近3ヶ月
母集団の推定（99.7%）
平均リターン: 0.1903% (worst: -0.0482%, best: 0.4288%)
標準偏差: 0.006126 (worst:0.008263, best:0.004817)
-----------------------------------------
モデルのスペック
シャープレシオ: 0.31062 (worst:-0.058289, b

Output()

In [130]:
en_dataset

In [131]:
import pandas as pd
pd.concat([lgbm_dataset.pred_result_df, en_dataset.pred_result_df], axis=1)

Target      Pred    Target  Pred
Date       Sector                                    
2014-01-06 ITインフラ -0.002487 -0.000062 -0.002487  42.0
           OA機器    0.008380  0.000325  0.008380  15.0
           その他工業   0.002106  0.000794  0.002106  34.0
           インフラ   -0.004196 -0.000266 -0.004196  17.0
           エンタメ    0.003894  0.000143  0.003894  40.0
...                     ...       ...       ...   ...
2024-10-18 電子部品         NaN -0.000339       NaN   2.0
           電気機器         NaN -0.000858       NaN  40.0
           電気機器2        NaN  0.000128       NaN  18.0
           非鉄金属         NaN  0.000564       NaN  48.0
           食品           NaN  0.001747       NaN  25.0

[126624 rows x 4 columns]

In [132]:
ml_dataset2 = MLDataset.MLDataset(f'{paths.ML_DATASETS_FOLDER}/LGBM_after_New48sectors.pkl.gz')

import pandas as pd
model = ml_dataset2.ml_models[0]
# モデルのトレーニング後に追加
importance_df = pd.DataFrame({
    'Feature': ml_dataset2.features_test_df.columns,
    'Importance': model.feature_importance(importance_type='gain')
}).sort_values(by='Importance', ascending=False)

pd.set_option('display.max_rows', None)
display(importance_df)
pd.reset_option('display.max_rows')

インスタンス情報を読み込みました。


,Feature,Importance
191,Sector_cat,97.721014
192,LASSO_pred,41.481591
185,21d_vola,3.653921
101,DJ_HouseholdHomeConst_1d_return,2.642363
40,IronORE62_1d_return,2.350221
186,21d_vola_rank,2.240175
179,5d_mom,2.185055
23,NaturalGasTR_1d_return,2.131033
174,EUR_1d_return,2.034184
5,MSCIEmerging_1d_return,1.935697
